In [11]:
import numpy as np
import pandas as pd

In [12]:
data = pd.read_csv('data.csv')
data_c = data[0:500]

data_c.select_dtypes(include='number').drop(['Unnamed: 0.1', 'Unnamed: 0'], axis=1).corr()
a_none = np.array(None)

In [13]:
'''Criando uma classe contendo algumas ferramentas para executar o least square'''
class LR():
    def __init__(self, x, y):
        self.x = x
        self.y = y
        self.predictions=a_none
        self.parametros=a_none

        
    def least_square(self):
        # Transpose na matrix com as variáveis
        row_x = self.x.T
        dot_x = row_x.dot(self.x)
        invert_dotx = np.linalg.inv(dot_x)
        self.parametros = invert_dotx.dot(row_x).dot(self.y)
        
        return invert_dotx.dot(row_x).dot(self.y)

    def prediction(self, x=a_none):
        if self.parametros.all() == None:
            self.parametros = self.least_square()

        if x.all() == None:
            self.predictions = self.x.dot(self.parametros)
            return self.x.dot(self.parametros)
        else:
            return x.dot(self.parametros)

    def MSE(self, y_hat=a_none, y=a_none): 
        # Quantia de resultados
        if y_hat.all() == None:
            self.predictions = self.prediction(self.x)
            self.mse = (self.predictions - self.y).T.dot((self.predictions - self.y)) / len(self.y)
            return self.mse
        
        else:
            return (y_hat - y).T.dot((y_hat - y)) / len(y)
    
    '''Uma das formas de tentar selecionar os melhores features
    do modelo, assim podemos diminuir a quantia de variáveis'''
    def Forward_selection(self, n_feature):
        k= 1
        # Manter as dimensão do data-set
        axis_x, axis_y = self.x.shape
        # Copiar a data-set pois serão feitas alterações nele
        x = self.x
        # A cada iteração a variável escolhido virá para cá
        select_feature = np.ones([axis_x, n_feature+1])

        for i in range(1, n_feature+1):

            predict = np.inf

            # Mantém a primeira coluna para o intercept e das variáveis já selecionadas
            feature = np.array(select_feature[:,:i+1])
            select = 0
         
            for j in range(1, axis_y-i+1):

                feature[:, i] = x[:, j]
                mse = LR(feature, self.y).MSE()
                # Encontra a variável que produziu menor mse
                if (mse < predict):
                    predict = mse
                    select = j
            
            # Variável escolhida é adicionada ao data-set das escolhas
            select_feature[:, i] = x[:, select]
            # Variável escolhida é excluída do data-set das buscas
            x = np.delete(x, select, axis=1)

        return select_feature
    

    def Backward_selection(self, n_feature):
        # n_feature demonstra quantia de feature deleted
        
        # Manter as dimensão do data-set
        axis_x, axis_y = self.x.shape
        # Copiar a data-set pois serão feitas alterações nele
        x = self.x
        # A quantia de feature que teremos no data-set
        select_feature = axis_y - n_feature

        for i in range(1, select_feature):
            predict = -np.inf
            select = 0
         
            for j in range(1, axis_y-i+1):

                feature = np.delete(x, j, axis=1)
                mse = LR(feature, self.y).MSE()
                # Encontra a variável que produziu maior mse
                if (mse > predict):
                    predict = mse
                    select = j
            
            # Variável com maior mse é excluída do data-set
            x = np.delete(x, select, axis=1)

        return x

In [16]:
# Selecionando a coluna da variável e prediction
np_data = data_c[['price', 'spec_rating']].to_numpy()

np_x = np_data.T[1]
np_y = np_data.T[0]

# Adicionando uma coluna com 1 em np_x
np_x = np.array([np.ones(500), np_x]).T

dim_1 = LR(np_x, np_y) 

# Parâmetros calculado pelo least squre
B = dim_1.least_square()
# Prevendo resultado
y_hat = dim_1.prediction()
# Mean Square Error testando a qualidade do modelo
mse = dim_1.MSE()
print(mse)

2457696370.0598817


In [17]:
'''A equação least square para mais de uma variável não é a mesma casos os dados
não sejam transformados, no least square há uma operação entre variáveis chamada
inner product que resultam em 0 casos os dados são ortogonais, assim iremos transforma-los
em ortogonais para aproveitar a mesma equação do caso com 1 dimensão'''

def residual(u, v):
    return u - (projection(u, v))

def projection(u, v):
    return (np.dot(u,v) / np.dot(v,v)) * v

# Função para deixar os dados ortogonais
def gram_schmidt(dataset):

    orthogonal = pd.DataFrame()
    colunas = dataset.columns

    for i in range(len(colunas)):
        v = dataset.iloc[:, i]
        for j in range(i):
            proj = projection(dataset.iloc[:, i], orthogonal.iloc[:, j])
            v = v - proj
        orthogonal[colunas[i]] = v 
    return orthogonal

In [22]:
# Data com mais de 1 variável
multi_data = data_c.select_dtypes(include='number').drop(['Unnamed: 0.1', 'Unnamed: 0'], axis=1)
multi_data

,price,spec_rating,display_size,resolution_width,resolution_height,warranty
0,49900,73.000000,15.6,1920.0,1080.0,1
1,39900,60.000000,15.6,1920.0,1080.0,1
2,26990,69.323529,14.0,1920.0,1080.0,1
3,59729,66.000000,14.0,2240.0,1400.0,1
4,69990,69.323529,13.3,2560.0,1600.0,1
...,...,...,...,...,...,...
495,59900,69.323529,15.6,1920.0,1080.0,1
496,55990,69.323529,14.0,1920.0,1080.0,1
497,19990,69.323529,15.6,1920.0,1080.0,1
498,47990,63.000000,14.0,2160.0,1440.0,1


In [23]:
multi_data  = gram_schmidt(multi_data)
# Os mesmos dados depois da ortogonalizados
multi_data

,price,spec_rating,display_size,resolution_width,resolution_height,warranty
0,49900,42.026128,-0.581678,-108.583224,-64.154312,-0.089732
1,39900,35.233317,2.294516,128.732709,28.460823,0.022862
2,26990,52.570327,-1.466991,113.034744,-58.454783,-0.012584
3,59729,28.925082,-0.558172,352.132971,92.852173,0.021250
4,69990,25.879415,-1.955616,612.913343,82.849719,0.013045
...,...,...,...,...,...,...
495,59900,32.142468,0.294784,-92.923241,-48.202254,-0.057162
496,55990,34.569479,-1.324436,-3.286348,-81.604057,-0.010939
497,19990,56.915359,0.098599,67.158648,-16.344064,-0.059426
498,47990,33.211701,0.059203,364.727842,202.802435,0.049160


In [19]:
# inner product entre as duas colunas resultando em um valor bem próximo de 0
sum(multi_data.iloc[:,4] * multi_data.iloc[:,2])

7.38766964269999e-11